In [1]:
import requests, gensim, spacy, json, nltk
from io import StringIO
import pandas as pd
import numpy as np
import re
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.decomposition import (
#     NMF, 
#     LatentDirichletAllocation,
#     TruncatedSVD
# )
# warnings.filterwarnings("ignore", category=DeprecationWarning)
# from sklearn.model_selection import GridSearchCV
# from nltk.tokenize import wordpunct_tokenize
# from nltk.metrics import ConfusionMatrix
# from nltk.tokenize import word_tokenize
# from nltk.probability import FreqDist
# from nltk.stem import PorterStemmer, WordNetLemmatizer
# import matplotlib.pyplot as plt
# from functools import reduce
# from time import time
# import warnings

D:\tiago\Downloads\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


## Bing news search

In [2]:
import requests

url = "https://bing-news-search1.p.rapidapi.com/news/search?mkt=pt-br"

querystring = {"q":"politica","freshness":"Month","textFormat":"Raw","safeSearch":"Off"}

headers = {
    'x-bingapis-sdk': "true",
    'x-rapidapi-host': "bing-news-search1.p.rapidapi.com",
    'x-rapidapi-key': "0388aaee5emsh59b4fba5041dd03p1959efjsn7e9a12071136"
    }

response = requests.get(url=url,\
                        headers=headers, params=querystring)

# everything_politics_payload = {'q': 'congresso, política', 'language': 'pt', 'sortBy': 'popularity',\
#                                'from':'2021-10-25'}
# response_politics = requests.get(url=everything_news_url,\
#                                  headers=headers, params=everything_politics_payload)
dict_bing = json.loads(json.dumps(response.json()))

print(response.text)

{"_type": "News", "readLink": "https:\/\/api.cognitive.microsoft.com\/api\/v7\/news\/search?q=politica", "queryContext": {"_type": "QueryContext", "originalQuery": "politica", "adultIntent": false}, "totalEstimatedMatches": 82800, "sort": [{"_type": "SortValue", "name": "Melhor correspondência", "id": "relevance", "isSelected": true, "url": "https:\/\/api.cognitive.microsoft.com\/api\/v7\/news\/search?q=politica"}, {"_type": "SortValue", "name": "Mais recente", "id": "date", "isSelected": false, "url": "https:\/\/api.cognitive.microsoft.com\/api\/v7\/news\/search?q=politica&sortby=date"}], "value": [{"_type": "NewsArticle", "name": "Mobilização contra a LGBTfobia ganha espaço na arena política", "url": "https:\/\/www.terra.com.br\/noticias\/mobilizacao-contra-a-lgbtfobia-ganha-espaco-na-arena-politica,6f1b3c6eaa5e90cb56e72cc6b6ec81canpke4c8d.html", "image": {"_type": "ImageObject", "thumbnail": {"_type": "ImageObject", "contentUrl": "https:\/\/www.bing.com\/th?id=OVFT.Al5HX32lm5F2M_C2H

In [3]:
print(dict_bing.keys())
print(dict_bing['queryContext'])
# print(dict_bing['totalEstimatedMatches'])
# print("sort:", dict_bing['sort'], "\n")

# -----------------------------

print(len(dict_bing['value']))

print(dict_bing['value'][0].keys(), "\n")
print(dict_bing['value'][0]['description'], "\n")

print("---------------------------")

print(dict_bing['value'][0], "\n")

dict_keys(['_type', 'readLink', 'queryContext', 'totalEstimatedMatches', 'sort', 'value'])
{'_type': 'QueryContext', 'originalQuery': 'politica', 'adultIntent': False}
10
dict_keys(['_type', 'name', 'url', 'image', 'description', 'provider', 'datePublished', 'category']) 

A mobilização dos 20 parlamentares que protocolaram, nesta quinta-feira, 28, uma ação contra o jogador de vôlei Maurício Souza por LGBTfobia é a mais recente de uma série de eventos que evidenciam a i 

---------------------------
{'_type': 'NewsArticle', 'name': 'Mobilização contra a LGBTfobia ganha espaço na arena política', 'url': 'https://www.terra.com.br/noticias/mobilizacao-contra-a-lgbtfobia-ganha-espaco-na-arena-politica,6f1b3c6eaa5e90cb56e72cc6b6ec81canpke4c8d.html', 'image': {'_type': 'ImageObject', 'thumbnail': {'_type': 'ImageObject', 'contentUrl': 'https://www.bing.com/th?id=OVFT.Al5HX32lm5F2M_C2Hf7K-C&pid=News', 'width': 700, 'height': 366}}, 'description': 'A mobilização dos 20 parlamentares que protoc

In [4]:
import requests

url = "https://bing-news-search1.p.rapidapi.com/news/search?mkt=pt-br"

querystring = {"safeSearch":"Off","textFormat":"Raw"}

headers = {
    'x-bingapis-sdk': "true",
    'x-rapidapi-host': "bing-news-search1.p.rapidapi.com",
    'x-rapidapi-key': "0388aaee5emsh59b4fba5041dd03p1959efjsn7e9a12071136"
    }

response = requests.get(url, headers=headers, params=querystring)

print(response.text)

{"_type": "News", "readLink": "https:\/\/api.cognitive.microsoft.com\/api\/v7\/news\/search?q=", "queryContext": {"_type": "QueryContext", "originalQuery": ""}, "value": [{"_type": "NewsArticle", "name": "Exame confirma que corpo desaparecido é de jovem visto pela última vez com policiais, em Paço do Lumiar", "url": "https:\/\/g1.globo.com\/ma\/maranhao\/noticia\/2021\/10\/30\/exame-confirma-que-corpo-desaparecido-e-de-jovem-visto-pela-ultima-vez-com-policiais-em-paco-do-lumiar.ghtml", "image": {"_type": "ImageObject", "thumbnail": {"_type": "ImageObject", "contentUrl": "https:\/\/www.bing.com\/th?id=OVFT.mv-vaaVrfBCcj85fKhI03C&pid=News", "width": 700, "height": 444}}, "description": "Jovem Marcelo Machado desapareceu após ser visto por policiais na Pindoba, em Paço do Lumiar — Foto: Reprodução\/TV Mirante Um exame realizado em um cadáver confirmou que o corpo é do jovem Marcelo Mac", "about": [{"_type": "Thing", "readLink": "https:\/\/api.cognitive.microsoft.com\/api\/v7\/entities\/98

In [5]:
import requests

url = "https://bing-news-search1.p.rapidapi.com/news"

querystring = {"mkt":"pt-br","safeSearch":"Off","textFormat":"Raw"}

headers = {
    'x-bingapis-sdk': "true",
    'x-rapidapi-host': "bing-news-search1.p.rapidapi.com",
    'x-rapidapi-key': "0388aaee5emsh59b4fba5041dd03p1959efjsn7e9a12071136"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"_type": "News", "webSearchUrl": "https:\/\/www.bing.com\/news\/search?q=top+stories&form=TNSA02", "value": [{"_type": "NewsArticle", "name": "Bolsonaro diz ter plano B para Auxílio Brasil caso Congresso não aprove PEC dos Precatórios", "url": "https:\/\/www.msn.com\/pt-br\/noticias\/brasil\/bolsonaro-diz-ter-plano-b-para-aux%C3%ADlio-brasil-caso-congresso-n%C3%A3o-aprove-pec-dos-precat%C3%B3rios\/ar-AAQ8czL", "description": "BRASÍLIA - Um dia após o novo secretário de Tesouro e Orçamento do Ministério da Economia, Esteves Colnago, ser categórico ao afirmar que a equipe econômica não tem um plano B para o pagamento do", "provider": [{"_type": "Organization", "name": "Estadão", "image": {"_type": "ImageObject", "thumbnail": {"_type": "ImageObject", "contentUrl": "https:\/\/www.bing.com\/th?id=ODF.yPPY_TD7Tyb66in3wa0DJQ&pid=news"}}}], "datePublished": "2021-10-30T15:15:00.0000000Z"}, {"_type": "NewsArticle", "name": "Governo deixa novos medicamentos para Covid fora do SUS e tenta salvar

In [6]:
print(dict_bing.keys())
print(dict_bing['queryContext'])
# print(dict_bing['totalEstimatedMatches'])
# print("sort:", dict_bing['sort'], "\n")

# -----------------------------

print(len(dict_bing['value']))
print(dict_bing['value'][0], "\n")

print(dict_bing['value'][1], "\n")

print(dict_bing['value'][2], "\n")

print(dict_bing['value'][3], "\n")

dict_keys(['_type', 'readLink', 'queryContext', 'totalEstimatedMatches', 'sort', 'value'])
{'_type': 'QueryContext', 'originalQuery': 'politica', 'adultIntent': False}
10
{'_type': 'NewsArticle', 'name': 'Mobilização contra a LGBTfobia ganha espaço na arena política', 'url': 'https://www.terra.com.br/noticias/mobilizacao-contra-a-lgbtfobia-ganha-espaco-na-arena-politica,6f1b3c6eaa5e90cb56e72cc6b6ec81canpke4c8d.html', 'image': {'_type': 'ImageObject', 'thumbnail': {'_type': 'ImageObject', 'contentUrl': 'https://www.bing.com/th?id=OVFT.Al5HX32lm5F2M_C2Hf7K-C&pid=News', 'width': 700, 'height': 366}}, 'description': 'A mobilização dos 20 parlamentares que protocolaram, nesta quinta-feira, 28, uma ação contra o jogador de vôlei Maurício Souza por LGBTfobia é a mais recente de uma série de eventos que evidenciam a i', 'provider': [{'_type': 'Organization', 'name': 'Terra', 'image': {'_type': 'ImageObject', 'thumbnail': {'_type': 'ImageObject', 'contentUrl': 'https://www.bing.com/th?id=ODF.a-

## NewsAPI

In [7]:
# Define API key and URLs for requests
headers = {'Authorization': '843f966d9f6d465bba288803803ee132'}
everything_news_url = 'https://newsapi.org/v2/everything'
sources_url = 'https://newsapi.org/v2/sources'

In [8]:
# headlines_payload = {'category': 'general', 'country': 'br, pt', "pageSize":100}
everything_politics_payload = {'q': 'congresso, política', 'language': 'pt', 'sortBy': 'popularity',\
                               'from':'2021-10-25', "pageSize":100}
everything_president_payload = {'q': "presidente, Bolsonaro", 'language': 'pt', 'sortBy': 'popularity',\
                               'from':'2021-10-25', "pageSize":100}
everything_economy_payload = {'q': "economia", 'language': 'pt', 'sortBy': 'popularity',\
                             'from':'2021-10-25', "pageSize":100}
everything_sports_payload = {'q': 'esporte', 'language': 'pt', 'sortBy': 'popularity',
                            'from':'2021-10-25', "pageSize":100}
sources_payload= {'category': 'general', 'language': 'pt'}
sports_sources_payload = {'category': 'sports', 'language': 'pt'}

In [9]:
resp_sources = requests.get(url = sources_url,\
                           headers = headers,\
                           params = sources_payload)
resp_sports_sources = requests.get(url = sources_url,\
                           headers = headers,\
                           params = sports_sources_payload)

In [10]:
response_politics = requests.get(url=everything_news_url,\
                                 headers=headers, params=everything_politics_payload)
response_president = requests.get(url=everything_news_url,\
                                  headers=headers, params=everything_president_payload)
response_economy = requests.get(url=everything_news_url,\
                                  headers=headers, params=everything_economy_payload)
response_sports = requests.get(url=everything_news_url,\
                                  headers=headers, params=everything_sports_payload)

In [11]:
dict_pol = json.loads(json.dumps(response_politics.json()))
dict_pres = json.loads(json.dumps(response_president.json()))
dict_econ = json.loads(json.dumps(response_economy.json()))
dict_sports = json.loads(json.dumps(response_sports.json()))

In [12]:
dicts = [dict_pol, dict_pres, dict_econ, dict_sports]

for d in dicts:
    for article in d['articles']:
        article['source_name'] = article['source']['name']

In [13]:
pol_articles = dict_pol['articles']
pres_articles = dict_pres['articles']
econ_articles = dict_econ['articles']
sports_articles = dict_sports['articles']

In [14]:
df_pol = pd.read_json(StringIO(json.dumps(pol_articles)))
df_pres = pd.read_json(StringIO(json.dumps(pres_articles)))
df_econ = pd.read_json(StringIO(json.dumps(econ_articles)))
df_sports = pd.read_json(StringIO(json.dumps(sports_articles)))

## Free News Newscatcher

In [15]:
import requests

url = "https://free-news.p.rapidapi.com/v1/search"

querystring = {"q":"politica","lang":"pt"}

headers = {
    'x-rapidapi-host': "free-news.p.rapidapi.com",
    'x-rapidapi-key': "0388aaee5emsh59b4fba5041dd03p1959efjsn7e9a12071136"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"status":"ok","total_hits":21,"page":1,"total_pages":1,"page_size":21,"articles":[{"title":"Painel: Servidores do TCU procuram ministros por projeto de lei que aumente sal\u00e1rios","author":null,"published_date":"2021-10-29 23:01:00","published_date_precision":"full","link":"https://www1.folha.uol.com.br/colunas/painel/2021/10/servidores-do-tcu-procuram-ministros-por-projeto-de-lei-que-aumente-salarios.shtml","clean_url":"uol.com.br","summary":"Representantes sindicais dos servidores do Tribunal de Contas da Uni\u00e3o t\u00eam procurado ministros da corte para convenc\u00ea-los a elaborar projeto de lei para que consigam um aumento de seus sal\u00e1rios para repor a infla\u00e7\u00e3o. Os ministros t\u00eam respondido que n\u00e3o h\u00e1 chance de sequer iniciar a discuss\u00e3o do tema enquanto houver teto de gastos. ORG XMIT: 343601_1.tif TCU. Brasilia DF julho 21: Predio do TCU, Tribunal de Contas da Uniao, na praca dos tribunais em 21. 07. 06 em Brasilia DF. As 16h00 politica.

## Contextual Web Search

In [16]:
import requests

url = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/search/NewsSearchAPI"

querystring = {"q":"taylor swift","pageNumber":"1","pageSize":"10","autoCorrect":"true","fromPublishedDate":"null","toPublishedDate":"null"}

headers = {
    'x-rapidapi-host': "contextualwebsearch-websearch-v1.p.rapidapi.com",
    'x-rapidapi-key': "0388aaee5emsh59b4fba5041dd03p1959efjsn7e9a12071136"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

{"_type":"news","didUMean":"","totalCount":905,"relatedSearch":[],"value":[{"id":"5040745161463234638","title":"Taylor Swift Collaborating With Selena Gomezs Ex?!","url":"https://www.youtube.com/watch?v=3HXoLfkPehE","description":"Are you ...Ready for It? Eagle-eyed fans are speculating that Taylor Swift may be collaborating with one of Selena Gomezs exes according to these receipts.W...","body":"Are you ...Ready for It? Eagle-eyed fans are speculating that Taylor Swift may be collaborating with one of Selena Gomezs exes according to these receipts.W...","snippet":"Are you ...Ready for It? Eagle-eyed fans are speculating that <b><b>Taylor Swift</b></b> may be collaborating with one of Selena Gomezs exes according to these ...","keywords":"","language":"en","isSafe":true,"datePublished":"2021-10-30T00:00:00","provider":{"name":"youtube","favIcon":"","favIconBase64Encoding":""},"image":{"url":"https://i.ytimg.com/vi/3HXoLfkPehE/maxresdefault.jpg","height":0,"width":0,"thumbnail":"","thum